In [39]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# This will automatically reload src/dtree.py every time you make changes and save the file
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
%matplotlib inline

In [49]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [50]:
df1 = pd.read_csv('./train.csv')
df2 = pd.read_csv('./test.csv')
l = df2.columns.tolist()
df1 = df1[l]
df = pd.concat([df1, df2])
trainLen = len(df1)

In [51]:
df1.info()
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16784 entries, 0 to 16783
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         16784 non-null  int64  
 1   title              16784 non-null  object 
 2   make               14624 non-null  object 
 3   model              16784 non-null  object 
 4   description        16439 non-null  object 
 5   manufactured       16590 non-null  float64
 6   original_reg_date  318 non-null    object 
 7   reg_date           16583 non-null  object 
 8   type_of_vehicle    16784 non-null  object 
 9   category           16784 non-null  object 
 10  transmission       16784 non-null  object 
 11  curb_weight        16205 non-null  float64
 12  power              14447 non-null  float64
 13  fuel_type          3490 non-null   object 
 14  engine_cap         16731 non-null  float64
 15  no_of_owners       16608 non-null  float64
 16  depreciation       163

In [31]:
# ['depreciation', 'omv', 'arf', 'age', 'remaining_coe', 'dereg_value', 'truck', 'suv', 'sports car', 'luxury sedan',\
#          'bus/mini bus', 'mid-sized sedan', 'stationwagon', 'hatchback', 'mpv']

# not sure: ['coe', 'road_tax', 'mileage']
# need to fill in: arf based on omv
# depreciation, ,

In [52]:
#fill in make
dfvalid = df.dropna(subset = ['make'])

missingMake = df.make.isnull()
dfm = df[missingMake]
missingModelAge = []
for idx, row in dfm.iterrows():
    model = row['model']
    missingModelAge.append(model)

mapper = {}
for idx, row in dfvalid.iterrows():
    model = row['model']
    make = row['make']
    mapper[model] = make

for idx, row in dfm.iterrows():
    model = row['model']
    df.at[idx, 'make'] = mapper[model]

print(df['make'].isnull().sum())

0


In [53]:
# get age and remaining coe
d1 = df['manufactured'].isnull()
df['age'] = df['manufactured'].copy()
df['age'][d1] = df['manufactured'][d1]
d1 = df['age'].isnull()
df['age'][d1] = df['reg_date'][d1]
d1 = df['age'].isnull()
df['age'][d1] = df['original_reg_date'][d1]
def month(x):
    if pd.isna(x):
        return np.nan
    mapper = {}
    all_m = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    for i in range(12):
        mapper[all_m[i]] = i + 1
    if len(str(x)) < 11:
        return (2021 - int(x)) * 12
    s = x.split('-')
    year = 2021 - int(s[-1])
    month = mapper[s[-2]]
    return year * 12 + month

def filterNeg(x):
    if x < 0:
        return 0
    return x
    
df['age'] = df['age'].map(month)

d1 = df['reg_date'].isnull()
df['remaining_coe'] = df['reg_date'].copy()
df['remaining_coe'][d1] = df['reg_date'][d1]
d1 = df['remaining_coe'].isnull()
df['remaining_coe'][d1] = df['original_reg_date'][d1]
d1 = df['remaining_coe'].isnull()
df['remaining_coe'][d1] = df['manufactured'][d1]
df['remaining_coe'] = 120 - df['remaining_coe'].map(month)
df['remaining_coe'] = df['remaining_coe'].map(filterNeg)

hasLifeSpan = ~df['lifespan'].isnull()
def get_remaining(x):
    diffFrom2021 = month(x)
    res = -diffFrom2021
    if res < 0:
        res = 0
    return res ##check 2021
df['remaining_coe'][hasLifeSpan] = df['lifespan'][hasLifeSpan].map(get_remaining)

G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
G:\softwares\envs\4277lab\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
G

In [54]:
# get omv
from collections import defaultdict
df_valid = df.dropna(subset = ['omv'])
# df_valid = pd.concat([df_valid, df_train], axis = 0)
# print(df_valid.shape)
omvs = df_valid['omv'].to_numpy()


# missingOmvModelsAge (1 year)
# missingomv = avg(modelAgeToSimilarOmv)
missingOmv = df.omv.isnull()
dfm = df[missingOmv]
missingModelAge = []
for idx, row in dfm.iterrows():
    model = row['model']
    age = row['age']
    p = (model, age)
    missingModelAge.append(p)

mapper = defaultdict(list)
for idx, row in df_valid.iterrows():
    model = row['model']
    age = row['age']
    omv = row['omv']
    p = (model, age)
    mapper[p].append(omv)

for idx, row in dfm.iterrows():
    model = row['model']
    age = row['age']
    p = (model, age)
    df.at[idx, 'omv'] = np.average(mapper[p])

G:\softwares\envs\4277lab\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
G:\softwares\envs\4277lab\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
G:\softwares\envs\4277lab\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
G:\softwares\envs\4277lab\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
G:\softwares\envs\4277lab\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
G:\softwares\envs\4277lab\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
G:\softwares\envs\4277lab\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning:

In [55]:
def divideDf(df):
    train = df[:trainLen]
    test = df[trainLen:]
    return len(train), train, test

def combineDf(train, test):
    return pd.concat([train, test])

In [56]:
trainLen, train, test = divideDf(df)

In [57]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16784 entries, 0 to 16783
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         16784 non-null  int64  
 1   title              16784 non-null  object 
 2   make               16784 non-null  object 
 3   model              16784 non-null  object 
 4   description        16439 non-null  object 
 5   manufactured       16590 non-null  float64
 6   original_reg_date  318 non-null    object 
 7   reg_date           16583 non-null  object 
 8   type_of_vehicle    16784 non-null  object 
 9   category           16784 non-null  object 
 10  transmission       16784 non-null  object 
 11  curb_weight        16205 non-null  float64
 12  power              14447 non-null  float64
 13  fuel_type          3490 non-null   object 
 14  engine_cap         16731 non-null  float64
 15  no_of_owners       16608 non-null  float64
 16  depreciation       163

In [58]:
# drop omv, record the omvIs for test

omvI = test[test.omv.isnull()].index.tolist()
print(len(omvI))

20


In [59]:
df = combineDf(train, test)
df = df.dropna(subset = ['omv'])
df.info()
# print(max(df.index))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21719 entries, 0 to 4999
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         21719 non-null  int64  
 1   title              21719 non-null  object 
 2   make               21719 non-null  object 
 3   model              21719 non-null  object 
 4   description        21263 non-null  object 
 5   manufactured       21468 non-null  float64
 6   original_reg_date  410 non-null    object 
 7   reg_date           21460 non-null  object 
 8   type_of_vehicle    21719 non-null  object 
 9   category           21719 non-null  object 
 10  transmission       21719 non-null  object 
 11  curb_weight        21030 non-null  float64
 12  power              18756 non-null  float64
 13  fuel_type          4537 non-null   object 
 14  engine_cap         21651 non-null  float64
 15  no_of_owners       21490 non-null  float64
 16  depreciation       2115

In [60]:
# age,power, enginecap
df[df.age < 0]['age'] = 0
df[df.age > 20000]['age'] = 0

G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [61]:
##train
print(df[df.engine_cap == 0])
df.at[8199, 'engine_cap'] = np.nan

      listing_id                                              title  \
8199     1005670  Mercedes-Benz E-Class E300e Plug-in Hybrid AMG...   

               make model                                        description  \
8199  mercedes-benz   amg  great deal! grab it! new facelifted 2021 with ...   

      manufactured original_reg_date     reg_date type_of_vehicle  \
8199        2021.0               NaN  16-jun-2021    luxury sedan   

                                               category  ...      omv  \
8199  parf car, almost new car, low mileage car, hyb...  ...  52492.0   

          arf  opc_scheme lifespan   eco_category  \
8199  66486.0         NaN      NaN  uncategorized   

                                               features  \
8199  2.0l m274 powerplant with eq boost producing 3...   

                                            accessories  indicative_price  \
8199  amg kits and wheels, memory seat, ambient ligh...               NaN   

      age  remaining_coe  
81

In [62]:
trianLen, train, test = divideDf(df)
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16784 entries, 0 to 44
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         16784 non-null  int64  
 1   title              16784 non-null  object 
 2   make               16784 non-null  object 
 3   model              16784 non-null  object 
 4   description        16439 non-null  object 
 5   manufactured       16594 non-null  float64
 6   original_reg_date  311 non-null    object 
 7   reg_date           16586 non-null  object 
 8   type_of_vehicle    16784 non-null  object 
 9   category           16784 non-null  object 
 10  transmission       16784 non-null  object 
 11  curb_weight        16240 non-null  float64
 12  power              14480 non-null  float64
 13  fuel_type          3497 non-null   object 
 14  engine_cap         16730 non-null  float64
 15  no_of_owners       16611 non-null  float64
 16  depreciation       16353 

In [63]:
df = combineDf(train, test)
df.to_csv('sure_clean_combined.csv')

In [64]:
df_valid = df.dropna(subset = ['power'])
ps = df_valid['power'].to_numpy()

missingPs = df.power.isnull()
dfm = df[missingPs]
missingPower = []
for idx, row in dfm.iterrows():
    model = row['model']
    missingPower.append(model)

mapper = defaultdict(list)
for idx, row in df_valid.iterrows():
    model = row['model']
    power = row['power']
    mapper[model].append(power)

for idx, row in dfm.iterrows():
    model = row['model']
    if len(mapper[model]):
        df.at[idx, 'power'] = np.average(mapper[model])

print(df['power'].isnull().sum())

1974


In [65]:
trainLen, train, test = divideDf(df)
train = train.dropna(subset = ['power', 'engine_cap'])
df = combineDf(train, test)

In [66]:
# coe: fill in missing value for coe according to: age and power
fill_arf = df[df.arf.isnull()]
for idx, row in fill_arf.iterrows():
    omv = row['omv']
    if omv < 20000:
        df.at[idx, 'arf'] = omv
    elif omv >= 20000 and omv <= 50000:
        df.at[idx, 'arf'] = (omv - 20000) * 1.40 + 20000
    else:
        df.at[idx, 'arf'] =  (omv - 50000) * 1.80 + 62000
print(len(df[df.arf.isnull()]))
# df = df[~(df.arf.isnull() & df.dereg_value.isnull())] # remove values whose dereg_value cannot be calculated

0


In [67]:
# for date missing both coe and dereg_value, record it's month engine and power
coe_to_search = []
to_search = df[df.coe.isnull() & df.dereg_value.isnull()]

for idx, row in to_search.iterrows():
    # category -> A / B
    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["age"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    coe_to_search.append((cat, month))

coe_to_search = list(set(coe_to_search))
print(len(coe_to_search))

24


In [68]:
t_to_coe = {}

for t in coe_to_search:
    t_to_coe[str(t)] = []

remains = df[~df.coe.isnull()]

for idx, row in remains.iterrows():
    if idx % 10000 == 0:
        print(idx)

    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["age"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    
    t = (cat, month)
    
    coe = row["coe"]
    if t in coe_to_search:
        t_to_coe[str(t)].append(coe)
        
c = 0
for idx, row in df.iterrows():
    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["age"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    t = str((cat, month))

    if type(row["coe"]) == float and pd.isna(row["coe"]):
        if t in t_to_coe:
            l = t_to_coe[t]
            if len(l) > 0:
                avg = sum(l) / len(l)
                df.at[idx, 'coe'] = avg
                c += 1
print(c)

0
1044


In [70]:
trainLen, train, test = divideDf(df)
train = train.dropna(subset=['coe'])
df = combineDf(train, test)

In [71]:
df.to_csv('sure_combined2.csv')

In [87]:
df = pd.read_csv('sure_combined2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20130 entries, 0 to 20129
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         20130 non-null  int64  
 1   listing_id         20130 non-null  int64  
 2   title              20130 non-null  object 
 3   make               20130 non-null  object 
 4   model              20130 non-null  object 
 5   description        19749 non-null  object 
 6   manufactured       19888 non-null  float64
 7   original_reg_date  394 non-null    object 
 8   reg_date           19878 non-null  object 
 9   type_of_vehicle    20130 non-null  object 
 10  category           20130 non-null  object 
 11  transmission       20130 non-null  object 
 12  curb_weight        19515 non-null  float64
 13  power              19701 non-null  float64
 14  fuel_type          3009 non-null   object 
 15  engine_cap         20115 non-null  float64
 16  no_of_owners       199

In [88]:
train = train.dropna(subset = ['depreciation'])
test.info()
trainLen = len(train)
print(trainLen)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3346 entries, 1639 to 4999
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         3346 non-null   int64  
 1   title              3346 non-null   object 
 2   make               3346 non-null   object 
 3   model              3346 non-null   object 
 4   description        3269 non-null   object 
 5   manufactured       3303 non-null   float64
 6   original_reg_date  66 non-null     object 
 7   reg_date           3304 non-null   object 
 8   type_of_vehicle    3346 non-null   object 
 9   category           3346 non-null   object 
 10  transmission       3346 non-null   object 
 11  curb_weight        3252 non-null   float64
 12  power              3057 non-null   float64
 13  fuel_type          700 non-null    object 
 14  engine_cap         3335 non-null   float64
 15  no_of_owners       3306 non-null   float64
 16  depreciation       32

In [89]:
import math
prices = pd.read_csv('train.csv')['price']
prices = prices[train.index]

for idx in range(len(train)):
    row = train.iloc[idx]
    depreciation = row['depreciation']
    age = row["age"]
    coe = row["coe"]
    omv = row["omv"]
    arf = row["arf"]
    remaining_coe = row['remaining_coe']

    min_parf = min(0.55 * omv, arf)
    if age <= 96:
        min_parf = 0.5 * arf ####
    elif age > 96 and age <= (2021 - 2008) * 12:
        min_parf = min(omv, arf)
    
    # print(idx, len(prices))
    new_coe = (prices.iloc[idx] - 0.5 * min_parf) / depreciation
    if new_coe <= 120 and new_coe >= 0 and age >= (2021 - 2002) * 12:
        train.iloc[idx]['remaining_coe'] = new_coe

train.info()

G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16345 entries, 0 to 1638
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         16345 non-null  int64  
 1   title              16345 non-null  object 
 2   make               16345 non-null  object 
 3   model              16345 non-null  object 
 4   description        16049 non-null  object 
 5   manufactured       16342 non-null  float64
 6   original_reg_date  41 non-null     object 
 7   reg_date           16345 non-null  object 
 8   type_of_vehicle    16345 non-null  object 
 9   category           16345 non-null  object 
 10  transmission       16345 non-null  object 
 11  curb_weight        16034 non-null  float64
 12  power              16208 non-null  float64
 13  fuel_type          2240 non-null   object 
 14  engine_cap         16341 non-null  float64
 15  no_of_owners       16339 non-null  float64
 16  depreciation       1634

In [ ]:
## redo test, need remaining coe
